Available as executable Jupyter Notebook:

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/HIL-HK/lets-plot-examples/master?filepath=remakes%2Fpushkin.ipynb)

## Preliminaries

In [ ]:
import pandas as pd
import geopandas as gpd

from lets_plot import *; LetsPlot.setup_html()
from lets_plot.settings_utils import maptiles_lets_plot;

In [ ]:
def set_stay_time(data):
    current_moves = moves_df[(moves_df.departure == data['name'])|(moves_df.arrival == data['name'])]
    years = current_moves.year.sort_values().unique()
    years_combined = []
    for year in years:
        if len(years_combined) == 0 or years_combined[-1][1] + 1 != year:
            years_combined.append((year, year))
        else:
            years_combined[-1] = (years_combined[-1][0], year)
    years_combined = [str(year_from) if year_from == year_to else '{0}-{1}'.format(year_from, year_to) \
                      for year_from, year_to in years_combined]
    data['name'] = '{0} ({1})'.format(data['name'], ', '.join(years_combined))
    return data

# Pushkin as Traveller

Pushkin grew up in Moscow and St. Petersburg. In 1819 he was exiled, at the age of nineteen, to a small town called Kishinev and then he was moved to his family estate in Ekaterinoslav. He remained in exile for nearly seven years until 1826 when Tsar Nicholas came to power. Below you can see a map of his travels throughout Russia.

<img src="https://raw.githubusercontent.com/HIL-HK/lets-plot-examples/master/data/pushkin/O.Kiprensky_-_A.S.Pushkin.jpg" alt="Portrait of A.S. Pushkin" style="width: 300px;"/>

[Original version](https://www.rgo.ru/ru/article/pushkin-puteshestvennik-linii-zhizni-linii-na-karte) of the graph that you can see below.

Also, see the [short journey through his biography](http://pushkinland.ru/2018/english/push1.php).

Some data was collected from here:

- [https://en.wikipedia.org/wiki/Alexander_Pushkin](https://en.wikipedia.org/wiki/Alexander_Pushkin)
- [http://hronika.su/pushkin-a-s](http://hronika.su/pushkin-a-s)
- [http://pushkin-art.ru/liveinhtml](http://pushkin-art.ru/liveinhtml)
- [http://alexanderpushkin.net/travels-in-russia](http://alexanderpushkin.net/travels-in-russia)

In [ ]:
places_df = pd.read_csv('https://raw.githubusercontent.com/HIL-HK/lets-plot-examples/master/data/pushkin/places.csv')
moves_df = pd.read_csv('https://raw.githubusercontent.com/HIL-HK/lets-plot-examples/master/data/pushkin/moves.csv')
df = moves_df.merge(places_df, left_on='departure', right_on='name')\
             .rename(columns=dict(longitude='from_lon', latitude='from_lat'))\
             .drop(columns=['name'])\
             .merge(places_df, left_on='arrival', right_on='name')\
             .rename(columns=dict(longitude='to_lon', latitude='to_lat'))\
             .drop(columns=['name'])
places_df = places_df.apply(set_stay_time, axis=1)

In [ ]:
ggplot() + \
    geom_livemap() + \
    geom_point(aes(x='longitude', y='latitude', fill='name'), data=places_df, size=3, color='black') + \
    geom_segment(aes(x='from_lon', y='from_lat', xend='to_lon', yend='to_lat', color='path'), data=df, \
                 arrow=arrow(type='closed', length=10, angle=15), size=1) + \
    scale_color_manual(name='Trip Name', values=['#addd8e', '#e34a33', '#8856a7', '#2c7fb8', \
                                                 '#1c9099', '#006d2c', '#fec44f', '#636363']) + \
    scale_fill_discrete(name='') + \
    coord_cartesian(xlim=[26, 58], ylim=[38, 62]) + \
    ggtitle("Alexander Pushkin's Trips") + \
    ggsize(1000, 700) + \
    theme(axis_title='blank', axis_text='blank', axis_ticks='blank', axis_line='blank')